# Data Processing Template

Since almost without exception the first step is working with data munging, this notebook is stubbed out with a very select few imports to speed up the initial steps following creating a project and copying in raw data.

In [1]:
# minimal module imports
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.features import FeatureLayer
from arcgis.geometry import Geometry
import pandas as pd
import os
import importlib

# import arcpy if available
if importlib.util.find_spec("arcpy") is not None:
    import arcpy

In [2]:
# a couple of handy variables and settings to get started

data = r'../data'
data_raw = os.path.join(data, 'raw')
data_ext = os.path.join(data, 'external')
data_int = os.path.join(data, 'interim')
data_out = os.path.join(data, 'processed')

gdb_ing = os.path.join(data_int, 'interim.gdb')
gdb_raw = os.path.join(data_raw, 'raw.gdb')

In [3]:
bp_pth = os.path.join(gdb_raw, 'block_points')
bg_pth = os.path.join(gdb_raw, 'block_groups')
cbsa_pth = os.path.join(gdb_raw, 'cbsas')

arcpy.env.overwriteOutput = True

In [4]:
bg_lst = [(bg_id, bg_geom) for bg_id, bg_geom in arcpy.da.SearchCursor(bg_pth, ['ID', 'SHAPE@'])]
bg_lst[:5]

[('410030103003', <Polygon object at 0x254a66106d8[0x254a651cb98]>),
 ('410110001001', <Polygon object at 0x254a655ff98[0x254a65e8a08]>),
 ('410110001002', <Polygon object at 0x254a655f6a0[0x254a65e89e0]>),
 ('410110001003', <Polygon object at 0x254b2ce1048[0x254a65e88c8]>),
 ('410110001004', <Polygon object at 0x254b2ce1080[0x254a65e8e18]>)]

In [6]:
bp_lst = [bp_geom for (bp_geom,) in arcpy.da.SearchCursor(bp_pth, 'SHAPE@')]

In [8]:
%%time
def get_weighted_centroid(bg_geom, bp_pth):
    local_bp_lst = arcpy.analysis.Clip(bp_pth, bg_geom, arcpy.Geometry())
    mean_pt_arcpy = arcpy.stats.MeanCenter(local_bp_lst, arcpy.Geometry())[0]
    mean_pt = Geometry(mean_pt_arcpy)
    return mean_pt

bg_geom = bg_lst[0]

get_weighted_centroid(bg_geom, bp_lst)

RuntimeError: Object: Error in executing tool

In [ ]:
bg_cent_lst = [(bg_id, get_weighted_centroid(blck_grp_geom)) for bg_id, blck_grp_geom in bg_lst]

print(f'Output Count: {len(bg_cent_lst)}')
bg_cent_lst[:5]

In [ ]:
df = pd.DataFrame(bg_cent_lst, columns=['block_group_id', 'SHAPE'])
df.spatial.set_geometry('SHAPE')
df.head()